In [ ]:
# downloading gcsfuse
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt -y -q update
!sudo apt -y -q install gcsfuse

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!mkdir -p data
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 waymo_open_dataset_v_1_2_0_individual_files data

In [ ]:
!pip install --upgrade pip
!pip install waymo-open-dataset-tf-2-4-0

In [ ]:
import tensorflow as tf
import os
from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils

In [ ]:
FILEPATH = {
    'train': './data/training',
    'test': './data/testing',
    'val': './data/validation',
}

In [ ]:
def getdataSet(filepath):
    dirs = os.listdir(filepath)
    dirs = [os.path.join(filepath,dir) for dir in dirs]

    raw_dataset = tf.data.TFRecordDataset(dirs)

    return raw_dataset

In [ ]:
dataSets = {}
for key in FILEPATH.keys():
    dataSets[key] = getdataSet(FILEPATH[key])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
HEIGHT, WIDTH = (640, 960)

In [ ]:
folder_path = {
    'train':'./train',
    'val':'./val'
}

for key in folder_path.keys():
    if not os.path.exists(folder_path[key]):
        os.mkdir(folder_path[key])

In [ ]:
def box2yoloFormat(center_x, center_y, length, width, ORIGIN_WIDTH, ORIGIN_HEIGHT):
    box_center_x = center_x / ORIGIN_WIDTH
    box_center_y = center_y / ORIGIN_HEIGHT
    box_width = length / ORIGIN_WIDTH
    box_height = width / ORIGIN_HEIGHT

    return (box_center_x, box_center_y, box_width, box_height)

In [ ]:
def imageDecode(image):
    
    image = tf.image.decode_jpeg(image)   

    h, w = image.shape[:2]

    image = tf.image.resize(image, (HEIGHT, WIDTH)) 
    images = tf.expand_dims(image, axis=0) / 255.0

    return images, h, w

In [ ]:
'''
enum Name {
    UNKNOWN = 0;
    FRONT = 1;
    FRONT_LEFT = 2;
    FRONT_RIGHT = 3;
    SIDE_LEFT = 4;
    SIDE_RIGHT = 5;
  }
'''
def createImgLabel(dataSetType):
    # SAVE FILE
    n = 0
    for record in dataSets[dataSetType]:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(record.numpy()))
        
        
        
        for i in range(5):
            camera_image = frame.images[i]
            if camera_image.name in [1,2,3]:
                if n%10 == 0:
                    print(n)
                if n <= 88378:
                    n = n + 1
                    continue

                # IMAGE
                images, ORIGIN_HEIGHT, ORIGIN_WIDTH = imageDecode(frame.images[i].image)
                plt.imsave( f'{folder_path[dataSetType]}/{n}.jpg', images[0].numpy())    
               
                # BOUNDING BOX
                for camera_labels in frame.camera_labels:
                    # Ignore camera labels that do not correspond to this camera.
                    if camera_labels.name != camera_image.name:
                        continue
                      
                    boxs = []
                    for label in camera_labels.labels:        
                        # BOX
                        box = label.box        
                        box_bound = box2yoloFormat(box.center_x, box.center_y, box.length, box.width, ORIGIN_WIDTH, ORIGIN_HEIGHT)
                        
                        # LABEL
                        box_label = label.type

                        boxs.append((box_label, *box_bound))
                    
                    with open(f'{folder_path[dataSetType]}/{n}.txt','w') as f:
                        box_str = []
                        for box in boxs:         
                            box_str.append(' '.join(str(x) for x in box))
                        f.write('\n'.join(box_str))
                
                n = n + 1
           
    

    

In [ ]:
def writeImgFileList(dataSetType):
    # WRITE train.txt
    with open(f'/content/waymo_object_detection/cfg/{dataSetType}.txt', 'w') as out:
        for img in [f for f in os.listdir(f'/content/waymo_object_detection/{dataSetType}') if f.endswith('jpg')]:
            out.write(f'/content/waymo_object_detection/{dataSetType}/' + img + '\n')

In [ ]:
createImgLabel('train')

In [ ]:
writeImgFileList('train')

In [ ]:
createImgLabel('val')

In [ ]:
!mv /content/drive/MyDrive/課程相關/109-2/自動駕駛實務/專案/Final/train/train/* /content/drive/MyDrive/課程相關/109-2/自動駕駛實務/專案/Final/train